<p>In questo notebook utilizzeremo un dataset di recensioni della piattaforma Yelp per esplorare alcune tecniche di preprocessing di testi per ottenere dati congrui all'applicazione di alcune tecniche di ML. Vedremo inoltre come è possibile visualizzare alcune caratteristiche dei dati e come sia possibile ridurre la dimensionalità del dataset tramite sampling qualora risulti difficile lavorare sull'intero dataset.
<p>Come prima cosa vediamo quindi di caricare i dati; quest'ultimi sono stati forniti da Yelp in formato <code>json</code> ma sono stati rielaborati ed esportati in formato <code>csv</code> per rendere più efficiente il loro caricamento tramite la libreria <code>pandas</code>.<br>
Il notebook assume che i dati siano contenuti in un file, <code>reviews.csv</code>, posto nella directory <code>./data/csv</code> relativamente alla root del progetto. In caso i dati siano presenti in un altra locazione sulla macchina o in un file con un diverso nome è necessario modificare le variabili <code>DATA_FOLDER</code> e <code>file_name</code>.

In [1]:
import os
import re

import sys
sys.path.append("..")

import time
import pickle
import graphviz
import numpy as np
import pandas as pd

from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer

from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import SGDRegressor, LinearRegression

from scripts.clean_results import clean_results_df, dump_results
from scripts.plotting_functions import export_tree_graph, render_tree_graph
from scripts.plotting_functions import plot_feature_importance, plot_coefficients_significance

from scripts.analyzers import simple_stemmer, regexp_stemmer

from bokeh.plotting import output_notebook, show
output_notebook()

SEED = 99
np.random.seed(SEED)

DATA_FOLDER = os.path.join(os.getcwd(), "../data")
RESULTS_FOLDER = os.path.join(DATA_FOLDER, "results")
CV_RESULTS_FOLDER = os.path.join(RESULTS_FOLDER, "csv")

Loading BokehJS ...

In [2]:
DATA_FOLDER = os.path.join(os.getcwd(), "../data/csv")
file_name = "reviews.csv"

reviews_fp = os.path.join(DATA_FOLDER, file_name)

<p>Jupyter permette l'eseguzione di comandi come da shell ponendo un punto escalamativo, <code>!</code>, prima del codice da interpretare come comando. In questo caso utilizzo il comando <code>head</code> per ottenere visualizzare l'output di un file, il parametro <code>-n</code> serve a specificare il numero di righe da stampare in output, nel nostro caso una. Notare come la prima riga del file non contenga dati, bensì i nomi delle colonne. Questa informazione è necessaria qualora si cerchi di creare un <code>DataFrame</code> Pandas leggendo direttamente un file csv. Infatti, qualora tale riga non sia presente, è necessario passare esplicitamente il parametro <code>names</code> contenente una lista di nomi all'invocazione della funzione. Tale lista sarà successivamente utilizzata da Pandas per assegnare dei nomi alle colonne. Nel nostro caso, essendo la riga presente, non è necessario fare nulla, Pandas è in grado di inferire i nomi corretti automaticamente.<br>

In [3]:
!head -n 1 ../data/csv/reviews.csv

review_id,business_id,user_id,stars,text,date


<p>Procediamo dunque alla creazione di un <code>DataFrame</code> leggendo i dati da file csv. Questa operazione è realizzata dalla funzione Pandas <code>read_csv</code> il cui primo parametro è un qualsiasi oggetto di tipo File oppure una stringa che rappresenta il path del file da leggere. Un altro parametro di particolare interesse per i nostri scopi è <code>index_col</code> che permette di specificare tra le altre cose il nome di una colonna da utilizzare come indice. Noi utilizzeremo l'id delle recensioni come colonna indice.

In [4]:
df = pd.read_csv(
    reviews_fp,
    index_col='review_id'
)

<p>Per visualizzare il contenuto di un <code>DataFrame</code> possiamo utilizzare il metodo <code>head(n=5)</code>. Questo metodo accetta un unico parametro, <code>n</code>, che permette di specificare il numero di righe da ritornare. Nel caso il parametro assuma valori negativi, e.g. <code>n=-5</code>, vengono ritornate tutte le righe meno le ultime cinque. Nel caso si voglia invece ottenere le ultime righe è presente il metodo <code>tail(n=5)</code>.
<p>Notare come il parametro <code>n</code> abbia un valore di default pari a cinque. Questo significa che la funzione, nonostante richieda un parametro, può essere invocata senza. In questo caso il valore del parametro sarà quello di default. Ogniqualvolta un parametro di una funzione o di un metodo sia seguito dal simbolo <code>=</code> e un valore sappiamo che tale valore è il valore di default del parametro.

In [5]:
df.head()

,business_id,user_id,stars,text,date
review_id,,,,,
i6g_oA9Yf9Y31qt0wibXpw,5JxlZaqCnk1MnbgRirs40Q,ofKDkJKXSKZXu5xJNGiiBQ,1,"Dismal, lukewarm, defrosted-tasting ""TexMex"" g...",2011-05-27 05:30:52
xQY8N_XvtGbearJ5X4QryQ,-MhfebM0QIsKt87iDN-FNw,OwjRMXRC0KyPrIlcjaXeFQ,2,"As someone who has worked with many museums, I...",2015-04-15 05:21:16
6TdNDKywdbjoTkizeMce8A,IS4cv902ykd8wj1TR0N3-A,UgMW8bLE0QMJDCkQ1Ax5Mg,4,"Oh happy day, finally have a Canes near my cas...",2017-01-14 21:56:57
L2O_INwlrRuoX05KSjc4eg,nlxHRv1zXGT0c0K51q3jDg,5vD2kmE25YBrbayKhykNxQ,5,This is definitely my favorite fast food sub s...,2013-05-07 07:25:25
ZayJ1zWyWgY9S_TRLT_y9Q,Pthe4qk5xh4n-ef-9bvMSg,aq_ZxGHiri48TUXJlpRkCQ,5,"Really good place with simple decor, amazing f...",2015-11-05 23:11:05


In [6]:
df.tail()

,business_id,user_id,stars,text,date
review_id,,,,,
AlYunCvl1CwUuOHdGbG92A,JDDZAflMR3oHBH5cKOQNpw,XGmokERSnIiHjt9qmlzMdQ,5,Just moved to the Phoenix area from Southern C...,2019-09-13 19:10:03
A_2inDTj_CdhfFYmy_0OJA,K5sUVFSGFEZosixSXgx5sw,X09mKkx416NmNfQIxlCgjg,1,Shake Shack started off pretty strong with mou...,2019-09-16 03:44:59
9Rm7MG9QNCaOpH7MukKaAw,5e6oNxr5ILU3FjctgvmUBA,fEnftKE6twLdX8HZOQBxDQ,1,I forgot to mention they're highly recommended...,2019-04-11 07:44:11
gIrjyOjIXrT4qzqhOby03w,jOMtqF2XnRgOZ2eOVLoxGA,iChmSxbAgP2DVvTcA_GDaA,2,I had the experience of dining at Soul Foo You...,2019-09-16 11:12:38
Oek3Z9jQ6YihW7PcJ63DQA,K6CLKaV__lrgejN-xLTS2w,Un77CNCyoO_mc430g9NZrw,1,I would give them zero stars if possible. Had ...,NaN


<p>Per conoscere la dimensione del DataFrame, e di conseguenza del data set, possiamo fare riferimento all'attributo <code>shape</code> della classe DataFrame. Esso ritorna una tupla contenente rispettivamente il numero di righe e il numero di colonne.

In [7]:
df.shape

(763379, 5)

<p>Il data set contiene circa otto milioni di recensioni ma un numero così elevato di osservazioni può creare problemi. Due aspetti molto importanti da tenere in considerazione sono infatti il consumo di memoria e i tempi di calcolo.<br>
Informazioni quali il consumo di memoria da parte di un DataFrame sono reperibili tramite il metodo <code>info()</code>.

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 763379 entries, i6g_oA9Yf9Y31qt0wibXpw to Oek3Z9jQ6YihW7PcJ63DQA
Data columns (total 5 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   business_id  763379 non-null  object
 1   user_id      763379 non-null  object
 2   stars        763379 non-null  int64 
 3   text         763379 non-null  object
 4   date         763378 non-null  object
dtypes: int64(1), object(4)
memory usage: 34.9+ MB


Notiamo che il data set consuma poco più di 300 MegaByte, non troppo per una macchina moderna che generalmente dispone di almeno 8 GigaByte di memoria primaria. I problemi legati al consumo di memoria diventano però rilevanti nelle fasi successive. Consideriamo ad esempio la costruzione di una bag of words e consideriamone la rappresentazione densa. Sappiamo che in tale rappresentazione ogni recensione avrà una prorpia riga e che quindi la matrice finale avrà 8 milioni di righe. Anche avendo un dizionario particolarmente ristretto, e.g. 10.000 parole, ci ritroveremmo con una matriche con 800 miliardi (credo) di celle. Assumendo che i dati contenuti nelle celle siano interi rappresentati su 64 bit (8 Byte) sarebbero necessari 625000000 GigaByte di memoria.<br>
Oltre agli eccessivi consumi di memoria, con una grande mole di dati si incorre in tempi di addestramento particolarmente elevati. 
<p>Per far fronte a questi problemi ricorriamo alla tecnica del sampling. Senza entrare troppo nei dettagli delle varie tecniche di sampling limitiamoci a quella più semplice : simple random sampling without replacement.<br>
L'idea dietro al sampling semplice senza rimpiazzamento è quella di estrarre casualmente dalla popolazione un qualche numero di osservazioni con l'importante dettaglio che, dopo l'estrazione di un elemento dalla popolazione, esso non venga reinserito nella popolazione stessa e quindi non possa essere estratto più di una volta.
<p>Un'operazione semplice come quella descritta sopra può essere ottenuta tramite il metodo <code>sample</code> della classe <code>DataFrame</code>.<br>
L'unico parametro che sfrutteremo è <code>n</code> che permette di specificare la dimensione della sample come numero di osservazioni in essa contenute ed è di conseguenza un intero. Alternativamete si può utilizzare il parametro <code>frac</code> che permette di specificare la frazione di elementi della popolazione originale da preservare ed è quindi un float compreso tra zero ed uno.

In [9]:
sample_size = 100000
sample_df = df.sample(n=sample_size,random_state=SEED)

<p>Qualora la popolazione originale sia particolarmente piccola o il processo di sampling particolarmente sfortunato potrebbe capitare che la sample ottenuta non sia rappresentativa delle caratteristiche della popolazione originale. Con il termine "rappresentativa" si intende che la distribuzione dei valori delle varie features nei due insiemi siano più o meno uguali. Per essere sicuri che la sample sia rappresentativa della popolazione con riferimento ad uno o più parametri può essere comodo generare una rappresentazione grafica della distribuzione dei parametri stessi nella sample e nella popolazione originale e metterle a confronto.<br>
Qualora si riscontri che la sample ottenuta non sia rappresentativa dell'originale potrebbe essere necessario utilizzare tecniche di sampling più elaborate quali ad esempio il sampling stratificato.<br>

<p>Per ora ci limitiamo a comparare la distribuzione della feature "stars" nella sample e nella popolazione originale utilizzando la libreria <code>Bokeh</code>. Per maggiori dettagli riguardo al funzionamento di Bokeh ti rimando al <a href="./Bokeh.ipynb">seguente notebook</a> o alla <a href="https://docs.bokeh.org/en/latest/">documentazione ufficiale</a>.<br>
Per rappresentare graficamente la distribuzione dei valori della colonna "stars" utilizzeremo un grafico a barre. Necessitiamo dunque di due informazioni : i possibili valori assumibili dalla feature e il numero di osservazioni associate ad ogni possibile valore.

<p>Per il calcolo delle informazioni di cui sopra sfrutteremo il metodo <code>groupby</code> della classe DataFrame. Questo metodo permette di raggruppare un DataFrame o una Series utilizzando un mapper, una funzione, una label o una lista di labels. Tale metodo accetta innumerevoli parametri di cui <code>by=</code> risulta di particolare interesse per i nostri scopi. Nel nostro caso passiamo al parametro il nome di una colonna con l'intento di specificare che i gruppi vadano creati sulla base dei valori di tale colonna. Il risultato di un'operazione di groupby è un oggetto della classe <code>DataFrameGroupBy</code> e non uno della classe <code>DataFrame</code> e di conseguenza supporta operazioni differenti. Per maggiori dettagli al riguardo è possibile fare riferimento alla <a href="https://pandas.pydata.org/pandas-docs/stable/reference/groupby.html">documentazione</a> o invocare la funzione built-in <code>help()</code> di Python nel seguente modo : <code>help(pd.core.groupby.DataFrameGroupBy)</code>.
    
<p>Sfrutteremo questo oggetto per calcolare il numero di osservazioni presenti in ogni gruppo; successivamente i valori verrano normalizzati nell'intervallo $[0,1]$ per evitare che la dimensione della popolazione sia presa in considerazione durante il confronto.

<p>Procediamo ora con la creazione degli oggetti DataFrameGroupBy per il DataFrame relativo all'intera popolazione e quello relativo alla sample ottenuta precedentemente.

In [10]:
df_groups = df.groupby(by='stars')
sample_groups = sample_df.groupby(by='stars')

<p>La classe <code>DataFrameGroupBy</code> supporta l'iterazione. Iterare sull'oggetto può auitare per avere un'idea più chiara di cosa sia contenuto nell'oggetto stesso. Il valore ritornato dell'iteratore è una tupla contenente il nome del grouppo e il gruppo stesso. Per poter osservare le singole entità ritorante dall'iteratore è quindi necessario 'spacchettare' i valori della tupla nel costrutto <code>for</code>.<br>
Vediamo un esempio :

In [11]:
#Ho commentato le funzioni di print per evitare che, nell'esportare il notebook in pdf, venisse
#incluso anche il risultato di tali operazioni. Per visualizzare l'output è sufficiente rimuovere
#i cancelletti
for group_name, group in sample_groups:
    #print("group_name : {}\n".format(group_name))
    #print("type(group) : {}\n".format(type(group)))
    #print("group :\n{}\n\n".format(group))
    pass

<p>Importante notare come i vari gruppi siano istanze della classe <code>DataFrame</code> e supportino di conseguenza tutte le operazioni relative a tale classe (e.g. metodi <code>groupby()</code>, <code>head()</code>, <code>tail()</code>, <code>info()</code> e attributi quali <code>shape</code>, <code>size</code>, <code>ndims</code>, etc...).
<p>In particolare è supportato il metodo <code>aggregate(func, axis=0)</code>, o l'alias <code>agg(func, axis=0)</code>, che permette di aggregare i dati utilizzando una o più operazioni lungo l'asse specificato.<br>
Il primo parametro può essere una funzione, una stringa, una lista o un dizionario. Dipendentemente dal tipo di dato del parametro passato si possono ottenere comportamenti differenti.<br>
Per i nostri scopi utilizzeremo un dizionario in cui le chiavi corrispondono ai nomi delle colonne sulle quali si vuole applicare la funzione mentre i valori corrispondono alle funzioni da applicare. Nel nostro caso la colonna lungo la quale vogliamo aggregare è la colonna 'stars' mentre la funzione che vogliamo utilizzare è la funzione <code>count()</code> che, come suggerisce il nome, conteggia gli elementi all'interno della colonna.<br>
Il secondo parametro, <code>axis=0</code> indica lungo che asse effettuare le aggregazioni. Di default il valore è 0, o "index", ad indicare che la funzione va applicata lungo le colonne ma è possibile applicare la funzione lungo le righe passando al parametro un valore pari ad 1 oppure alla stringa "columns".

In [12]:
full_counts = df_groups.agg({"stars":"count"})
sample_counts = sample_groups.agg({"stars":"count"})

In [13]:
full_counts

,stars
stars,
1,124076
2,59542
3,77659
4,156238
5,345864


In [14]:
sample_counts

,stars
stars,
1,16277
2,7853
3,10097
4,20366
5,45407


<p>Ottenute le informazioni riguardo al numero di recensioni per ogni possibile valore della feature "stars" possiamo procedere nel trasformare i valori ottenuti in valori percentuali. Questa operazione è necessaria affinchè la popolosità della sample o del dataset originale non venga tenuta in considerazione durante il confronto.

In [15]:
df_size = df.shape[0]
sample_counts = sample_counts['stars'].map(lambda x : (x*100)/sample_size)
full_counts = full_counts['stars'].map(lambda x : (x*100)/df_size)

In [16]:
full_counts

stars
1    16.253525
2     7.799795
3    10.173060
4    20.466636
5    45.306984
Name: stars, dtype: float64

In [17]:
sample_counts

stars
1    16.277
2     7.853
3    10.097
4    20.366
5    45.407
Name: stars, dtype: float64

<p>Procediamo ora con la creazione del grafico.<br>
Per prima cosa è necessario istanziare un oggetto della classe <code>Bokeh.plotting.figure</code>; questa classe rappresenta la "tela" su cui successivamente sarà possibile "dipingere" i grafici.<br>
Il costruttore di tale classe accetta innumerevoli parametri per configurare i più svariati aspetti quali : altezza, larghezza, titolo, range degli assi, tipi di asse, etc... Per maggiori dettagli consultare la <a href="http://docs.bokeh.org/en/1.3.4/docs/reference/plotting.html">documentazione</a>.

In [ ]:
fig = plot_pop_v_sample_stars(full_counts, sample_counts)

In [21]:
show(fig)

<p>Il grafico è abbastanza semplice, è possibile migliorarlo aggiungendo ad esempio una leggenda, delle lables per facilitare la lettura delle altezze, modificare i ticks sugli assi, etc.<br>
Per ora teniamo le cose come sono.

In [22]:
df['stars'].mean()

3.707737572031717

In [23]:
sample_df['stars'].mean()

3.70773

<h3>Distribuzione della lunghezza delle recensioni</h3>

In [24]:
df['words'] = df['text'].apply(lambda x : x.split(" "))
df['word_count'] = df['words'].apply(lambda x : len(x))

In [25]:
counts_grs = df.groupby(by='word_count')
counts = counts_grs.agg({'word_count':'count'})
counts = counts.rename(columns={'word_count':'freq'})
counts

,freq
word_count,
1,155
2,52
3,53
4,91
5,129
...,...
1087,1
1105,1
1191,1


In [50]:
mean_ = df['word_count'].mean()
mode_ = df['word_count'].mode().values[0]
median_ = df['word_count'].median()
std_ = df['word_count'].std()
mean_

110.22898979406035

In [49]:
fig = plot_reviews_length(counts, median_, std_)
show(fig)

In [45]:
spearmanr(df['word_count'], df['stars'])

SpearmanrResult(correlation=-0.23886882473837903, pvalue=0.0)

<p>Analizzando i dati si osserva che non tutte le recensioni sono espresse in lingua inglese. Prima di poter utilizzare i dati è dunque necessario pulirli da queste incongruenze.<br>

<p>Per riconoscere la lingua di un testo sono disponibili svariate librerie, noi utilizzeremo <a href="https://pypi.org/project/langdetect/">langdetect</a>.<br>
Procediamo quindi con il creare una nuova colonna del dataset contenente il valore <code>np.nan</code> che è utile per identificare dati mancanti. Successivamente popoliamo la colonna <code>'lang'</code> con i risultati dell'applicazione della funzione di language detection sulla colonna <code>'text'</code>.

<p>Per maggiori dettagli : <br>
<ul>
    <li><a href="https://stackoverflow.com/questions/39142778/python-how-to-determine-the-language">Domanda StackOverflow su come riconoscere la lingua di una stringa</a></li>
</ul>

In [ ]:
def _detect(text):
    try:
        l = detect(text)
    except:
        l = np.nan
    finally:
        return l
    
sample_df['lang'] = sample_df['text'].apply(_detect)

<p>Per sapere quali siano le lingue in cui sono espresse le recensioni possiamo utilizzare il metodo <code>unique()</code> della classe <code>pd.Series</code>. Questo metodo ritorna un array contenente i valori unici presenti nella Series su cui è applicato, nel nostro caso la colonna <code>'lang'</code>.

In [ ]:
sample_df['lang'].unique()

<p>Le recensioni sembrerebbero espresse nelle lingue più svariate.<br>
Vediamo più nel dettaglio cosa abbiamo ottenuto. Possiamo ad esempio utilizzare il seguente codice per farci ritornare le righe del DataFrame <code>sample_df</code> tali per cui la colonna <code>'lang'</code> contenga la stringa 'it'.

In [ ]:
sample_df[sample_df['lang'] == 'it']

<p>Sembrerebbe che la libreria che abbiamo utilizzato venga ingannata da recensioni contenenti partole italiane classificando erroneamente recensioni inglesi come italiane. Vediamo però anche che alcune recensioni sono effettivamente in italiano!<br>
Utilizzando una tecnica simile a quella impiegata sopra possiamo farci ritornare un DataFrame contenente le righe del DataFrame originale tali per cui la lingua sia diversa dall'inglese.<br>
Vediamo cosa otteniamo : 

In [ ]:
sample_df[sample_df['lang'] != 'en']

<p>Di nuovo semprerebbe che alcune recensioni in lingua inglese vengano erroneamente classificate come espresse in una lingua differente, notiamo però che generalmente l'assegnamento linguistico è abbastanza coerente.
<p>Vediamo ora di eliminare dal DataFrame tutte quelle recensioni che non siano espresse in inglese.

In [ ]:
sample_df = sample_df[sample_df['lang'] == 'en']

In [ ]:
sample_df.head()

In [ ]:
sample_df['lang'].unique()

<p>Un altra importante operazione della fase di pulizzia è la gestione dei valori mancanti. Verifichiamo quindi se vi siano elementi del data set aventi valori mancanti. Per farlo sfruttiamo la il metodo <code>isnull()</code> della classe DataFrame che ci ritorna un DataFrame booleano in cui ogni cella presenta <code>True</code> se essa conteneva un valore NA. Valori NA possono essere : <code>None</code> e <code>np.nan</code>.
<p>Successivamente possiamo utilizzare il metodo <code>any</code> sul DataFrame ritornato dall'invocazione di <code>isnull</code> per verificare che colonne contengano valori NA.

In [ ]:
sample_df.isnull().any()

<p>A quanto pare nessuna delle colonne contiene valori NA, procediamo dunque con la creazione di un insieme di test. Questo insieme verrà tenuto da parte per validare la capacità di generalizzazione del modello che decideremo di utilizzare. E' prassi comunque utilizzare circa un 80% dei dati come insieme di training e un 20% per l'insieme di test. Per effettuare questa suddivisione utilizzeremo la funzione <code>train_test_split</code> della libreria <code>scikit-learn</code>.

In [ ]:
train, test = train_test_split(
    sample_df,
    test_size=0.2,
    random_state=SEED,
    stratify=sample_df["stars"]
)

X_train, Y_train = train['text'], train['stars']
X_test, Y_test = test['text'], test['stars']

In [ ]:
train_fp = os.path.join(os.getcwd(), "../data/csv/train.csv")
train.to_csv(
    train_fp
)

In [ ]:
test_fp = os.path.join(os.getcwd(), "../data/csv/test.csv")
test.to_csv(
    test_fp
)